In [1]:
import os
import openai


In [2]:
openai.api_key = os.environ.get('OPENAI_KEY')
# openai.Model.list()

In [3]:
response = openai.Completion.create(
    engine="davinci", 
    prompt="I expect", 
    max_tokens=5
)
print(response["choices"][0]["text"])

 they'll be coming for


In [4]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt="Write a tagline for a business called gistai, that specializes in creating personal dialog agents."
)

In [5]:
print(response)

{
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-7uJWd2k9pYnRmBLhREDyGJt4ky7lh",
  "object": "text_completion",
  "created": 1693655779,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\n\"Gistai: Your Conversation Partner for the Digital Age\"",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 19,
    "completion_tokens": 15,
    "total_tokens": 34
  }
}


In [6]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

In [7]:
print(response)

{
  "id": "chatcmpl-7uJWfdur25iyJkm1zYznl9M1gw5Ma",
  "object": "chat.completion",
  "created": 1693655781,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The World Series in 2020 was played at Globe Life Field in Arlington, Texas."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 53,
    "completion_tokens": 18,
    "total_tokens": 71
  }
}


In [8]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
      {"role": "system", "content": "You are a customer service representative."},
      {"role": "user", "content": "Hi, I have a problem with my account."}
  ]
)


In [9]:
reply = response.choices[0]["message"]["content"]
print(reply)

I'm sorry to hear that. I'll do my best to assist you. Can you please provide me with more details about the issue you're facing?


In [10]:
from dataset.msc_sessions import MSC_Session

basedir = "/Users/FrankVerhoef/Programming/PEX/data/msc/msc_dialogue/"
MSC_Session.set(sessionbreak_token='<sessionbreak>')
msc_session = MSC_Session(basedir=basedir, session=2, subset='train', include_history=True, include_persona=True, augmented=True)

2023-09-02 13:56:25,558 INFO     | Read 4000 dialogues from MSC session 2 for train dataset


In [11]:
ROLE = {
    'sessionbreak': 'system',
    'you': 'user',
    'me': 'assistant'
}
OPENAI_MODEL = "gpt-3.5-turbo"

def get_speaker_mapping(history):

    def find(l, x):
        i = 0
        while i < len(l):
            if l[i] == x:
                break
            i += 1
        return i

    mapping = {"Speaker 1": "me", "Speaker 2": "you", "Nobody": "sessionbreak"}
    speakers = [speaker for speaker, _ in history]

    # First check if history contains sessionbreaks
    first_sessionbreak = find(speakers, 'Nobody')
    if first_sessionbreak >= len(speakers):
        persona_sentences = []
        dialogue_turns = history
        
        # No sessionbreaks --> determine mapping based on last speaker_label
        if len(speakers) > 0 and speakers[-1] == 'Speaker 1':
            mapping = {"Speaker 1": "you", "Speaker 2": "me", "Nobody": "sessionbreak"}

    else:
        if first_sessionbreak + 1 < len(speakers):

            # Now filter out all utterances until the next sessionbreak (because these are persona sentences)
            next_sessionbreak = first_sessionbreak + 1 + find(speakers[first_sessionbreak + 1:], 'Nobody')
            assert next_sessionbreak < len(speakers), "Should contain sessionbreak for start of session"
            persona_sentences = history[first_sessionbreak:next_sessionbreak]
            dialogue_turns = history[next_sessionbreak:]
            speakers = [s for s in speakers[next_sessionbreak:] if s != 'Nobody']

            # Update mapping if last speaker is speaker 1
            if len(speakers) > 0 and speakers[-1] == 'Speaker 1':
                mapping = {"Speaker 1": "you", "Speaker 2": "me", "Nobody": "sessionbreak"}

    return mapping, persona_sentences, dialogue_turns

def openai_messages(history):

    mapping, persona_sentences, dialogue_turns = get_speaker_mapping(history)

    messages = []
    if len(persona_sentences) > 0:
        my_character = [s for p, s in persona_sentences if mapping[p] == 'me']
        user_character = [s for p, s in persona_sentences if mapping[p] == 'you']
        if len(my_character) > 0:
            messages.append({"role": "system", "content": "I am an assistant with the following persona: " + ' '.join(my_character)})
        else:
            messages.append({"role": "system", "content": "A sessionbreak indicates I have not spoken with the user for a while. If I have chatted with the user before, I use all information that I have about the user to follow up on previous conversations"})
        if len(user_character) > 0:
            messages.append({"role": "system", "content": "The user has the following persona: " + ' '.join(user_character)})
        else:
            messages.append({"role": "system", "content": "I have no information about the user"})

    messages.extend([
        {"role": ROLE[mapping[speaker]], "content": content}
        for speaker, content in dialogue_turns
    ])
    return messages

def openai_response(history):

    response = openai.ChatCompletion.create(
        model=OPENAI_MODEL,
        messages=openai_messages(history)
        )
    print(response)
    return response.choices[0]["message"]["content"]

In [12]:
msc_session.history[0]

[('Nobody', 'personas'),
 ('Speaker 1', "I'm a perfectionist."),
 ('Speaker 1', "If things aren't done right I'll redo them again and again."),
 ('Speaker 1',
  'I take forever to get tasks done so I start early and clock out late.'),
 ('Speaker 1', 'I work too much.'),
 ('Speaker 1', 'I think I need a vacation.'),
 ('Speaker 1',
  "I've been working a lot of extra hours. I want to break from my non-stop work."),
 ('Speaker 1', 'I like going to the beach.'),
 ('Speaker 1', 'I love brownies.'),
 ('Speaker 2', "I served or serve in the military. I've traveled the world."),
 ('Speaker 2', "I've blown things up."),
 ('Speaker 2', "I've never been to Bora Bora."),
 ('Speaker 2', 'I love chocolate.'),
 ('Nobody', '2 days ago'),
 ('Speaker 1',
  'I need some advice on where to go on vacation, have you been anywhere lately?'),
 ('Speaker 2', "I have been all over the world. I'm military."),
 ('Speaker 1', 'That is good you have alot of travel experience'),
 ('Speaker 2',
  'Sure do. And a lot 

In [13]:
openai_messages(msc_session.history[0])

[{'role': 'system',
  'content': "I am an assistant with the following persona: I'm a perfectionist. If things aren't done right I'll redo them again and again. I take forever to get tasks done so I start early and clock out late. I work too much. I think I need a vacation. I've been working a lot of extra hours. I want to break from my non-stop work. I like going to the beach. I love brownies."},
 {'role': 'system',
  'content': "The user has the following persona: I served or serve in the military. I've traveled the world. I've blown things up. I've never been to Bora Bora. I love chocolate."},
 {'role': 'system', 'content': '2 days ago'},
 {'role': 'assistant',
  'content': 'I need some advice on where to go on vacation, have you been anywhere lately?'},
 {'role': 'user', 'content': "I have been all over the world. I'm military."},
 {'role': 'assistant',
  'content': 'That is good you have alot of travel experience'},
 {'role': 'user',
  'content': 'Sure do. And a lot of experience 

In [14]:
msc_session.next_utterance[0]

('Speaker 1', 'Are you still in the military?')

In [15]:
l = [1, 2, 3, 4]
l[2:2]

[]

In [16]:
openai_response(msc_session.history[1])
    

{
  "id": "chatcmpl-7uJWml2EuJpWcfM9uzqx3leSVnQXb",
  "object": "chat.completion",
  "created": 1693655788,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Yes, I am an AI assistant working on behalf of the military. How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 380,
    "completion_tokens": 21,
    "total_tokens": 401
  }
}


'Yes, I am an AI assistant working on behalf of the military. How can I assist you today?'